This script analyses the pointwise convergence at r=0, for near critical simulations (the threshold of black hole formation). The echoes refer to the behaviour of the scalar field at r=0, near criticallity. The script assumes that you have run the same simulation for at least three resolutions. It is suggested that you run with an infalling outer boundary rmax, but you could also use if with a fixed rmax. You need to save the r0data type (classical variables around r=0).

In principle, you could use this script both for classical and semiclassical simulations. Keep in mind though it would be sensible to do so only if backreaction is included in the semiclassical case (otherwise the result is exactly the same as in classical). In this case, the infalling rmax does not perform well, so you would probably need to run with a fix rmax and then you need a lot of points to see an echo. Maybe it is not computationally viable.

In [ ]:
# load packages
using HDF5
using LaTeXStrings
using Plots; pythonplot()
using DelimitedFiles
using Interpolations

using SpheriCo

In [ ]:
# give the directory where the data from all the runs are saved
base = "../examples/classical_runs/"
# id1
par = "a0.1680164_b0.0_c1.0_rmax6.015_tmax6.0_cfl0.125_sigma0.02_infalling_rmax_true_rand_false_overMp2_25.132741228718345_damp0"
# id2
#par = "a0.0449_b5.0_c1.0_rmax11.76_tmax11.75_cfl0.125_sigma0.02_infalling_rmax_true_rand_false_overMp2_25.132741228718345_damp0"

dir = base*par
# convention for 1/M_P^2 (Planck mass)
oMp2 = 25.132741228718345

# directory where plots are saved
out_dir = "./critical/"*par
if ispath(out_dir)==false
    mkpath(out_dir)
end

Below you can see how many timesteps of r0data have been saved in each simulation. r0data are the classical data (classical state vector) only for the points r[1:5], where r[3]=0 and r[1],r[2] are ghost points. We save more points such that we can take derivatives in postprocessing if we need.

In [ ]:
# choose the D that corresponds to your coarse resolution
D = 5
Nr = (2^D)*128 + 3# the overal course graining

# list all available iterations (and corresponding files)

# coarse resolution
n = 0
(its_c, all_filenames_c) = list_h5_files(dir*"/data_$((Nr-3)*2^n+3)", prefix="r0data_")
println("# timesteps saved in coarse resolution: ", length(its_c))

# medium
n = 1
(its_m, all_filenames_m) = list_h5_files(dir*"/data_$((Nr-3)*2^n+3)", prefix="r0data_")
println("# timesteps saved in medium resolution: ", length(its_m))

# fine
n = 2
(its_f, all_filenames_f) = list_h5_files(dir*"/data_$((Nr-3)*2^n+3)", prefix="r0data_")
println("# timesteps saved in fine resolution: ", length(its_f))

Save the data at r0 for the lapse, scalr field, Ricci scalar, Hamiltonian and momentum constraints, as well as the difference between the metric functions A,B and the components of the extrinsic curvature KA, KB. This is done for all three resolutions. Save the corresponding timesteps as well.

In [ ]:
# coarse, n=0
n = 0

α_r0_t_c         = zeros(length(its_c));
Φ_r0_t_c         = zeros(length(its_c));
R_r0_t_c         = zeros(length(its_c));
H_r0_t_c         = zeros(length(its_c));
P_r0_t_c         = zeros(length(its_c));
AminusB_r0_t_c   = zeros(length(its_c));
KAminusKB_r0_t_c = zeros(length(its_c));

t_list_c = zeros(length(its_c));

for i in 1:length(its_c)
    it = its_c[i]
    it_str  = lpad(it, 4, "0")
    
    #v_classic_labels = ["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]
    v = h5read(dir*"/data_$((Nr-3)*2^n + 3)/r0data_$(it_str).h5","v")
    Φ  = v[:,1]
    A  = v[:,4]
    B  = v[:,5]
    K  = v[:,8]
    KB = v[:,9]
    α  = v[:,11]
    r = h5read(dir*"/data_$((Nr-3)*2^n + 3)/r0data_$(it_str).h5","r")
    dr = r[4] - r[3]
    t = h5readattr(dir*"/data_$((Nr-3)*2^n + 3)/r0data_$(it_str).h5", "/")["time"]
    H, P = SpheriCo.classical.constraints(v, dr, oMp2);
    R  = SpheriCo.Ricci_scalar(v, r, oMp2);
        
    α_r0_t_c[i] = α[3]
    Φ_r0_t_c[i] = Φ[3]
    R_r0_t_c[i] = R[3]
    H_r0_t_c[i] = H[3]
    P_r0_t_c[i] = P[3]
    AminusB_r0_t_c[i] = A[3]-B[3]
    KAminusKB_r0_t_c[i] = K[3]-3*KB[3]
    
    t_list_c[i] = t
end

In [ ]:
# medium, n=1
n = 1

α_r0_t_m         = zeros(length(its_m));
Φ_r0_t_m         = zeros(length(its_m));
R_r0_t_m         = zeros(length(its_m));
H_r0_t_m         = zeros(length(its_m));
P_r0_t_m         = zeros(length(its_m));
AminusB_r0_t_m   = zeros(length(its_m));
KAminusKB_r0_t_m = zeros(length(its_m));

t_list_m = zeros(length(its_m));

for i in 1:length(its_m)
    it = its_m[i]
    it_str  = lpad(it, 4, "0")
    
    #v_classic_labels = ["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]
    v = h5read(dir*"/data_$((Nr-3)*2^n + 3)/r0data_$(it_str).h5","v")
    Φ  = v[:,1]
    A  = v[:,4]
    B  = v[:,5]
    K  = v[:,8]
    KB = v[:,9]
    α  = v[:,11]
    r = h5read(dir*"/data_$((Nr-3)*2^n + 3)/r0data_$(it_str).h5","r")
    dr = r[4] - r[3]
    t = h5readattr(dir*"/data_$((Nr-3)*2^n + 3)/r0data_$(it_str).h5", "/")["time"]
    H, P = SpheriCo.classical.constraints(v, dr, oMp2);
    R  = SpheriCo.Ricci_scalar(v, r, oMp2);
        
    α_r0_t_m[i] = α[3]
    Φ_r0_t_m[i] = Φ[3]
    R_r0_t_m[i] = R[3]
    H_r0_t_m[i] = H[3]
    P_r0_t_m[i] = P[3]
    AminusB_r0_t_m[i] = A[3]-B[3]
    KAminusKB_r0_t_m[i] = K[3]-3*KB[3]
    
    t_list_m[i] = t
end

In [ ]:
# fine, n=2
n = 2

α_r0_t_f         = zeros(length(its_f));
Φ_r0_t_f         = zeros(length(its_f));
R_r0_t_f         = zeros(length(its_f));
H_r0_t_f         = zeros(length(its_f));
P_r0_t_f         = zeros(length(its_f));
AminusB_r0_t_f   = zeros(length(its_f));
KAminusKB_r0_t_f = zeros(length(its_f));

t_list_f = zeros(length(its_f));

for i in 1:length(its_f)
    it = its_f[i]
    it_str  = lpad(it, 4, "0")
        
    #v_classic_labels = ["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]
    v = h5read(dir*"/data_$((Nr-3)*2^n + 3)/r0data_$(it_str).h5","v")
    Φ  = v[:,1]
    A  = v[:,4]
    B  = v[:,5]
    K  = v[:,8]
    KB = v[:,9]
    α  = v[:,11]
    r = h5read(dir*"/data_$((Nr-3)*2^n + 3)/r0data_$(it_str).h5","r")
    dr = r[4] - r[3]
    t = h5readattr(dir*"/data_$((Nr-3)*2^n + 3)/r0data_$(it_str).h5", "/")["time"]
    H, P = SpheriCo.classical.constraints(v, dr, oMp2);
    R  = SpheriCo.Ricci_scalar(v, r, oMp2);
    
    α_r0_t_f[i] = α[3]
    Φ_r0_t_f[i] = Φ[3]
    R_r0_t_f[i] = R[3]
    H_r0_t_f[i] = H[3]
    P_r0_t_f[i] = P[3]
    AminusB_r0_t_f[i] = A[3]-B[3]
    KAminusKB_r0_t_f[i] = K[3]-3*KB[3]
    
    t_list_f[i] = t
end

It is common that there is some offset at the timesteps between the different resolutions. Below, you can check how big this is. You have to manually set the indices in the time lists that you want to compare. The cell below prints only the second result. You can comment it out to see the other result, or use a println() command.

In [ ]:
t_list_c[1:6114] - t_list_m[1:6114]
t_list_m[1:4145] - t_list_f[1:4145]

Calculate the proper time at r=0, for the various resolutions.

In [ ]:
# find proper time at r=0
# coarse
dt_c = t_list_c[2]-t_list_c[1];
proper_time_list_c = zeros(length(t_list_c));
for i in 2:length(proper_time_list_c)
    proper_time_list_c[i] = proper_time_list_c[i-1] + α_r0_t_c[i]*dt_c
end

# medium
dt_m = t_list_m[2]-t_list_m[1];
proper_time_list_m = zeros(length(t_list_m));
for i in 2:length(proper_time_list_m)
    proper_time_list_m[i] = proper_time_list_m[i-1] + α_r0_t_m[i]*dt_m
end

# fine
dt_f = t_list_f[2]-t_list_f[1];
proper_time_list_f = zeros(length(t_list_f));
for i in 2:length(proper_time_list_f)
    proper_time_list_f[i] = proper_time_list_f[i-1] + α_r0_t_f[i]*dt_f
end

We want to find similarity time, which is useful in the plots to see better the behaviour at r=0. For this, we need two pairs of consequtive zero crossings. We find them for all three resolutions below.

In [ ]:
# coarse
test_sign = sign(Φ_r0_t_c[1])
zero_crossings_c = []
for i in 1:length(proper_time_list_c)
    if test_sign != sign(Φ_r0_t_c[i])
        append!(zero_crossings_c, proper_time_list_c[i])
        test_sign = sign(Φ_r0_t_c[i])
    end
end
println(zero_crossings_c)

# medium
test_sign = sign(Φ_r0_t_m[1])
zero_crossings_m = []
for i in 1:length(proper_time_list_m)
    if test_sign != sign(Φ_r0_t_m[i])
        append!(zero_crossings_m, proper_time_list_m[i])
        test_sign = sign(Φ_r0_t_m[i])
    end
end
println(zero_crossings_m)

# fine
test_sign = sign(Φ_r0_t_f[1])
zero_crossings_f = []
for i in 1:length(proper_time_list_f)
    if test_sign != sign(Φ_r0_t_f[i])
        append!(zero_crossings_f, proper_time_list_f[i])
        test_sign = sign(Φ_r0_t_f[i])
    end
end
println(zero_crossings_f)

Set the similarity time to be used in the plots. The result might be very different for different resolutions, so choose with care (probably trial and error). The accumulation time is when the oscillation of the scalar field at r=0 starts to happen (fast).

In [ ]:
# set the zero_crossings and proper time to be used for the caclulation of the accumulation time
zero_crossings = zero_crossings_c
proper_time_list = proper_time_list_c

println("the zero corssings used are: ", zero_crossings)
# choose the pairs of consequtive zero crossings
un = zero_crossings[1]
unplus1 = zero_crossings[2]
um = zero_crossings[3]
umplus1 = zero_crossings[4]
# evaluate the formula for accumulation time
accum_time = (un*umplus1 - unplus1*um)/(un - unplus1 - um + umplus1)
println("the accumulation time is: ", accum_time)
# find the similarity time T = -ln(u*-u), where u is proper time at r=0 and u* accumulationtime
similarity_time = []
i = 1
while proper_time_list[i] < accum_time
    T = -log(accum_time - proper_time_list[i])
    append!(similarity_time, T)
    i += 1
end

Plots of various quantities at r=0, against the similarity time. You need to choose tf by hand and make sure that it is available in all resolutions.

In [ ]:
similarity_time

In [ ]:
ti = 2
tf = 2511 #id1
#tf =  3634 #id2 
tii = similarity_time[ti]
tff = similarity_time[tf]

plot(similarity_time[ti:tf], Φ_r0_t_c[ti:tf], title = "r=0", 
    label=L"Φ_5", linewidth=2.6)#,  color = "royalblue")
plot!(similarity_time[ti:tf], Φ_r0_t_m[ti:tf],
    label=L"Φ_6", linewidth=2.8,  style= :dash)#,  color = "coral")
p1 = plot!(similarity_time[ti:tf], Φ_r0_t_f[ti:tf],
    label=L"Φ_7", linewidth=3,  style = :dot, legend =:topright, frame=true,
    legendfontsize=10, grid=true, xlim=(tii,tff))
#color = "darkgreen",
plot!([0.6], seriestype="hline", label="", linewidth=1, color = "black", style = :dashdot)
p1 = plot!([-0.6], seriestype="hline", label="", linewidth=1, color = "black", style = :dashdot,
xticks=([0,2,4,6],["","","",""]), # id1
#xticks=([-2,0,2,4],["","","",""]), # id2
yticks=([0.6,0.3,0,-0.3,-0.6],["0.6","0.3","0","-0.3","-0.6"]),
legendfontsize=10,
xtickfont=10, xguidefontsize=10,
ytickfont=10, yguidefontsize=10,)

plot(similarity_time[ti:tf], abs.(R_r0_t_c[ti:tf]), label=L"|R_5|", linewidth=2.6)#,  color = "royalblue")
plot!(similarity_time[ti:tf], abs.(R_r0_t_m[ti:tf]), label=L"|R_6|", linewidth=2.8,  style= :dash)#,  color = "coral")
p2 = plot!(similarity_time[ti:tf], abs.(R_r0_t_f)[ti:tf], label=L"|R_7|", linewidth=3,  style = :dot,# color = "darkgreen",
    legend =:bottomright, frame=true, legendfontsize=10, grid=true, xlim=(tii,tff),
yscale=:log10,
xticks=([0,2,4,6],["","","",""]), # id1
#xticks=([-2,0,2,4],["","","",""]), # id2
yticks=([1e-3,1,1e3,1e6],[L"10^{-3}",L"1",L"10^3",L"10^6"]), #id1
#yticks=([1e-5,1e-1,1e2,1e5],[L"10^{-5}",L"10^{-1}",L"10^2",L"10^5"]), #id2
#ylim = (1e-7,1e6), # id2
xtickfont=10, xguidefontsize=10,
ytickfont=10, yguidefontsize=10,
)

plot(similarity_time[ti:tf], abs.(H_r0_t_c[ti:tf]), label=L"|H_5|", linewidth=2.6)#,  color = "royalblue")
plot!(similarity_time[ti:tf], abs.(H_r0_t_m[ti:tf]), label=L"|H_6|", linewidth=2.8,  style= :dash)#,  color = "coral")
p3 = plot!(similarity_time[ti:tf], abs.(H_r0_t_f[ti:tf]), label=L"|H_7|", linewidth=3,  style = :dot, #color = "darkgreen",
    legend =:bottomright, frame=true, legendfontsize=10, grid=true, xlim=(tii,tff),
yscale=:log10,
xticks=([0,2,4,6],["","","",""]), #id1
#xticks=([-2,0,2,4],["","","",""]), # id2
yticks=([1e-5,1e-2,10,1e4],[L"10^{-5}",L"10^{-2}",L"10",L"10^{4}"]), #id1
ylim = (1e-5,1e5), #id1
#yticks=([1e-7,1e-4,1e-1,1e2],[L"10^{-7}",L"10^{-4}",L"10^{-1}",L"10^{2}"]), #id2
#ylim = (1e-7,1e5), #id2
xtickfont=10, xguidefontsize=10,
ytickfont=10, yguidefontsize=10,
)

plot(similarity_time[ti:tf], abs.(P_r0_t_c[ti:tf]), label=L"|P_5|", linewidth=2.6)#,  color = "royalblue")
plot!(similarity_time[ti:tf], abs.(P_r0_t_m[ti:tf]), label=L"|P_6|", linewidth=2.8,  style= :dash)#,  color = "coral")
p4 = plot!(similarity_time[ti:tf], abs.(P_r0_t_f)[ti:tf], label=L"|P_7|", linewidth=3,  style = :dot, #color = "darkgreen",
    legend =:bottomright, frame=true, legendfontsize=10, grid=true, xlim=(tii,tff),
yscale=:log10,
xticks=([0,2,4,6],["","","",""]), #id1
#xticks=([-2,0,2,4],["","","",""]), # id2
yticks=([1e-5,1e-2,10,1e4],["","","",""]), #id1
ylim = (1e-5,1e5), #id1
#yticks=([1e-7,1e-4,1e-1,1e2],["","","",""]), #id2
#ylim = (1e-7,1e5), #id2
xtickfont=10, xguidefontsize=10,
ytickfont=10, yguidefontsize=10,
)

plot(similarity_time[ti:tf], abs.(AminusB_r0_t_c[ti:tf]),
    label=L"|A_5-B_5|", linewidth=2.6)#,  color = "royalblue")
plot!(similarity_time[ti:tf], abs.(AminusB_r0_t_m[ti:tf]),
    label=L"|A_6-B_6|", linewidth=2.8,  style= :dash)#,  color = "coral")
p5 = plot!(similarity_time[ti:tf], abs.(AminusB_r0_t_f)[ti:tf],
    label=L"|A_7-B_7|", linewidth=3,  style = :dot, #color = "darkgreen",
    legend =:topright, frame=true, legendfontsize=10, grid=true, xlim=(tii,tff),
xlabel="T",
yscale=:log10,
xticks=([0,2,4,6],["0","2","4","6"]), #id1
#xticks=([-2,0,2,4],["-2","0","2","4"]), # id2
yticks=([1e-7,1e-5,1e-3,1e-1],[L"10^{-7}",L"10^{-5}",L"10^{-3}",L"10^{-1}"]),
ylim = (1e-7,10),
xtickfont=10, xguidefontsize=10,
ytickfont=10, yguidefontsize=10,
)

plot(similarity_time[ti:tf], abs.(KAminusKB_r0_t_c[ti:tf]), 
    label=L"|K_{A5}-K_{B5}|", linewidth=2.6)#,  color = "royalblue")
plot!(similarity_time[ti:tf], abs.(KAminusKB_r0_t_m[ti:tf]),
    label=L"|K_{A6}-K_{B6}|", linewidth=2.8,  style= :dash)#,  color = "coral")
p6 = plot!(similarity_time[ti:tf], abs.(KAminusKB_r0_t_f)[ti:tf],
    label=L"|K_{A7}-K_{B7}|", linewidth=3,  style = :dot, #color = "darkgreen",
    legend =:bottomright, frame=true, legendfontsize=10, grid=true, xlim=(tii,tff),
xlabel="T",
yscale=:log10,
xticks=([0,2,4,6],["0","2","4","6"]), #id1
#xticks=([-2,0,2,4],["-2","0","2","4"]), # id2
yticks=([1e-7,1e-5,1e-3,1e-1],["","","",""]),
ylim = (1e-7,10),
xtickfont=10, xguidefontsize=10,
ytickfont=10, yguidefontsize=10,
)

plt = plot(p1, p2, p3, p4, p5, p6, 
    layout = grid(3, 2),
    wsize = (750,530))

savefig(plt, out_dir*"/r0_plots_D$(D)_D$(D+1)_D$(D+2).pdf")

Below is the pointwise convergence of the scalar field and lapse function at r=0. 

In [ ]:
ti = 2
tf = 2511 # id1
#tf =  3634 #id2 

tii = similarity_time[ti]
tff = similarity_time[tf]

plot(similarity_time[ti:tf], α_r0_t_c[ti:tf], 
    label=L"\alpha_5", linewidth=2.6)#,  color = "royalblue")
plot!(similarity_time[ti:tf], α_r0_t_m[ti:tf],
    label=L"\alpha_6", linewidth=2.8,  style= :dash)#,  color = "coral")
p7 = plot!(similarity_time[ti:tf], α_r0_t_f[ti:tf],
    label=L"\alpha_7", linewidth=3,  style = :dot, legend =:topright, frame=true,
    legendfontsize=10, grid=true, xlim=(tii,tff),
xticks=([0,2,4,6],["","","",""]), #id1
#xticks=([-2,0,2,4],["","","",""]), # id2
yticks=([0,0.25,0.5,0.75,1],["0","0.25","0.5","0.75","1"]),
xtickfont=10, xguidefontsize=10,
ytickfont=10, yguidefontsize=10,
)

plot(similarity_time[ti:tf], abs.(Φ_r0_t_c[ti:tf].-Φ_r0_t_m[ti:tf]), 
    label=L"|Φ_{56}|", linewidth=2.8)#,  color = "royalblue")
p8 = plot!(similarity_time[ti:tf], 4*abs.(Φ_r0_t_m[ti:tf]-Φ_r0_t_f[ti:tf]),
        label=L"4\,|Φ_{67}|", linewidth=3,  style= :dash,#  color = "coral", 
    frame=true, legend =:bottomright, legendfontsize=10, grid=true, xlim=(tii,tff), xlabel="T",
xticks=([0,2,4,6],["0","2","4","6"]), #id1
#xticks=([-2,0,2,4],["-2","0","2","4"]), # id2
yticks=([1e-9,1e-6,1e-3,1],[L"10^{-9}",L"10^{-6}",L"10^{-3}",L"1"]),
ylim = (1e-9,1),
xtickfont=10, xguidefontsize=10,
ytickfont=10, yguidefontsize=10,
yscale=:log10)

plot(similarity_time[ti:tf], abs.(α_r0_t_c[ti:tf].-α_r0_t_m[ti:tf]), frame = true,
    label=L"|α_{56}|", linewidth=2.8)#,  color = "royalblue")
p9 = plot!(similarity_time[ti:tf], 4*abs.(α_r0_t_m[ti:tf].-α_r0_t_f[ti:tf]),
    label=L"4\,|α_{67}|", linewidth=3,  style= :dash, # color = "coral",
legend =:bottomright, legendfontsize=10, grid=true, xlim=(tii,tff), xlabel="T",
xticks=([0,2,4,6],["0","2","4","6"]), #id1
#xticks=([-2,0,2,4],["-2","0","2","4"]), # id2
yticks=([1e-9,1e-6,1e-3,1],["","","",""]),
ylim = (1e-9,1),
xtickfont=10, xguidefontsize=10,
ytickfont=10, yguidefontsize=10,
yscale=:log10
)

plt = plot(p1, p7, p8, p9, 
    layout = grid(2, 2),
    wsize = (750,375))

savefig(plt, out_dir*"/r0_conv_D$(D)_D$(D+1)_D$(D+2).pdf")